In [ ]:
#### To-Do: 
# Work on the Hover
# Put all visualizations within one script
# Implememt Box Charts
# Style the Quantity of Hot Days with split color background

In [202]:
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
import pandas as pd
import sqlite3

output_notebook()
#Gets the distinct years from the database
conn = sqlite3.connect('weather.db')
c = conn.cursor()

####### retrieves the highest year per max temp
df = pd.read_sql('Select * from weather where station = "USC00154958"', conn)

#df.loc[df.groupby(['MONTH', 'DAY'])['TMAX'].idxmax()].to_csv('maxyears2.csv')
temp_uphi = df.loc[df.groupby(['MONTH', 'DAY'])['TMAX'].idxmax()]
#temp_uplo = df.loc[df.groupby(['MONTH', 'DAY'])['TMAX'].idxmin()]
#temp_dhi = df.loc[df.groupby(['MONTH', 'DAY'])['TMIN'].idxmax()]
temp_dlo = df.loc[df.groupby(['MONTH', 'DAY'])['TMIN'].idxmax()]

temp_uhi_data = temp_uphi.groupby('YEAR').size()
#temp_ulo_data = temp_uplo.groupby('YEAR').size() * -1
#temp_dhi_data = temp_dhi.groupby('YEAR').size()
temp_dlo_data = temp_dlo.groupby('YEAR').size() * -1

# hover = HoverTool()
# hover.tooltips = [
#     ("date", "$index"),
#     ("(x,y)", "($x, $y)"),
#     ("radius", "@radius"),
#     ("fill color", "$color[hex, swatch]:fill_color"),
#     ("foo", "@foo"),
#     ("bar", "@bar"),
# ]
rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
p = figure(width=1000, plot_height=500)
p.xgrid.grid_line_color=None
p.ygrid.grid_line_color='red'
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Years'
p.yaxis.axis_label = "Num of Lowest Lows \t|| \tNum of Highest Highs"
p.line(temp_uhi_data.index, temp_uhi_data.values, line_width=2, color='red', legend="Max Highs")
#p.line(temp_ulo_data.index, temp_ulo_data.values, line_width=2, color='blue', legend="Max Lows")
#p.line(temp_dhi_data.index, temp_dhi_data.values, line_width=2, color='green', legend="Min Highs")
p.line(temp_dlo_data.index, temp_dlo_data.values, line_width=2, color='blue', legend="Min Lows")

p.tools.append(hover)
show(p)

Loading BokehJS ...

In [207]:
from bokeh.layouts import gridplot, column
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.plotting import figure
import pandas as pd
import sqlite3

###
### This is the collection and manipulation of data
###
output_notebook()
#Gets the distinct years from the database
conn = sqlite3.connect('weather.db')
c = conn.cursor()
####### retrieves the highest year per max temp
weather = pd.read_sql('select DATE, (TMAX+TMIN)/2 Temp, TMAX high, TMIN low, YEAR, MONTH, DAY from weather where station = "USW00093821"', conn)
last_year = pd.read_sql('select MONTH, DAY, (TMAX+TMIN)/2 temp from weather where YEAR = "2017"', conn).groupby('MONTH')
this_year = pd.read_sql('select MONTH, DAY, (TMAX+TMIN)/2 temp from weather where YEAR = "2018"', conn).groupby('MONTH')
# This gets the temperature from SQL as standard and splits them into month groups
avg_month_temp = pd.read_sql("Select Month, Year, AVG(TMAX) high, AVG(TMIN) low, AVG((TMAX+TMIN)/2) temp from weather group by 1, 2", conn)
#avg_month_temp2 = pd.read_sql("Select Month, Year, AVG((TMAX+TMIN)/2) temp from weather group by 1, 2", conn)
month_group = avg_month_temp.groupby('MONTH')

# Graph ranges
max_temp, min_temp = (weather.high.max() + 10), weather.low.min()

#Get the years for max temperatures per each calender day
weather2 = weather.loc[weather.groupby(['MONTH', 'DAY'])['Temp'].idxmax()]
hottest_years_by_day = weather2.groupby('YEAR').size()

# The function to create the AVG_MONTH_TEMP by YEAR
def temp_plots(month, data):
    # Names for Titles
    month_names = {
                  1:"January",
                  2: "February",
                  3: "March",
                  4: "April",
                  5: "May",
                  6: "June",
                  7: "July",
                  8: "August",
                  9: "September",
                  10: "October",
                  11: "November",
                  12: "December"
                  }
    #Hover tools stuff- yet to be properly implimented
#     hover = HoverTool()
#     hover.tooltips = [
#         ("date", "$index"),
#         ("(x,y)", "($x, $y)"),
#         ("radius", "@radius"),
#         ("fill color", "$color[hex, swatch]:fill_color"),
#         ("foo", "@foo"),
#         ("bar", "@bar"),
#     ]
    # rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    # mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    p = figure(width=500, plot_height=250, y_range=(min_temp, max_temp), title = month_names[month])
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Years'
    p.yaxis.axis_label = "Temperature"
    
    p.line(data.YEAR, data.high, line_width=2, color='red', legend="yearly high temp")
    p.line(data.YEAR, data.low, line_width=2, color='blue', legend="yearly low temp")
        
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.padding = 1

    p.tools.append(hover)

    return p
    
# Grouping 
def data_grid(lst, col=2):
    output = []
    for i in range(0, len(lst), col):
        if (i + (col - 1)) >= len(lst):
            output.append(lst[i:])
        else:
            output.append(lst[i:(i+(col))])
    return output

Loading BokehJS ...

In [212]:
years = []
for name, group in month_group:
    years.append(temp_plots(name, group))
show(column(Div(text="<h2>Quantity of Hottest Days</h2>")))
show(p)
show(column(Div(text="<h2>Months Temp by Year</h2>")))
show(gridplot(data_grid(years)))
#data_grid(years, 3)

In [215]:
weather2.quantile(q=0.25)

Temp       63.5
high       71.0
low        55.0
YEAR     2007.0
MONTH       4.0
DAY         8.0
Name: 0.25, dtype: float64